In [1]:
import os
import re
import logging
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text

In [2]:
def initialiseConnection():
    logging.disable(logging.WARNING)
    usr = 'root'
    pwd = 'Barney2015$$'
    host = 'localhost'
    port = 3306
    dbName = 'amazonscraping'
    engine = create_engine(f'mysql+mysqldb://{usr}:{pwd}@{host}:{port}/{dbName}', future=True)
    return engine

In [3]:
def retrieveTable(table):
    engine = initialiseConnection()
    with engine.connect() as conn:
        df = pd.read_sql(text(f'SELECT * FROM {table};'), conn)
    return df

In [4]:
def addNewItem(url, name, type):
    try:
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        })
        response = session.get(url)
        htmlContent = response.text
        soup = BeautifulSoup(htmlContent, 'html.parser')
        priceDiv = soup.find('div', class_='a-section a-spacing-none aok-align-center aok-relative')
        price = None
        if priceDiv:
            price = float(priceDiv.text.strip().split(' ')[-1][1:])
        df = pd.DataFrame([[name, type, url, price, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]], columns=['Name', 'Type', 'Link', 'Price', 'Time'])
        engine = initialiseConnection()
        df.to_sql(name='amazonprices', con=engine, index=False, if_exists='append')
    except:
        print('Invalid Amazon URL Supplied')

In [5]:
retrieveTable('amazonprices')

,Name,Type,Link,Price,Time
0,Uncanny X-Force,Omnibus,https://www.amazon.com.au/UNCANNY-X-FORCE-REME...,167.95,2024-08-01 15:54:10
1,Uncanny X-Force,Omnibus,https://www.amazon.com.au/UNCANNY-X-FORCE-REME...,167.95,2024-08-01 15:56:05
2,Lego Avengers Tower,Omnibus,https://www.amazon.com.au/LEGO-76269-Avengers-...,NaN,2024-08-01 15:57:43
3,Uncanny X-Force,Omnibus,https://www.amazon.com.au/UNCANNY-X-FORCE-REME...,167.95,2024-08-01 15:58:48
4,Punisher Max By Ennis Vol. 1,Omnibus,https://www.amazon.com.au/PUNISHER-GARTH-ENNIS...,121.01,2024-08-01 16:48:27
5,Uncanny X-Force By Remender,Omnibus,https://www.amazon.com.au/UNCANNY-X-FORCE-REME...,167.95,2024-08-01 21:57:10


In [6]:
uncannyUrl = 'https://www.amazon.com.au/UNCANNY-X-FORCE-REMENDER-OMNIBUS-PRINTING/dp/1302957732/ref=sr_1_2?crid=18Z2HWOAF2CRG&dib=eyJ2IjoiMSJ9.YOTNRks2nVm1tZp3ZXhRsEobHBCpOn2n2AUyU2-6lj3UrBjDv__BeQDzx1LIwu7NUNLmMYz4FOCrWU7gL_7T-Q3jfqj48sOrjUm9NY8Q-BCZSmgIev2sH7vaALE1HlQWUx9mzXVdBcx9xRVZI3dRgSnd5OlOSmgAryl1nrpHDBDW6u4q0BwibAfNfXOa39h6Q0NvWx_94lN7dpIBj5Dr2S62U3u1GMgiyC6dnfhtOPkZhnEERLg1iClY-hNr4-uv16m3esL0wzZ4Cbu4GW3DtRx0A2SBm-Sx-47xhNlIqSs.KpEzAxCepp6jYEcUP3aXvKNRAW_-UL4ITs21J13JdmA&dib_tag=se&keywords=uncanny+x+force+omnibus&qid=1722520554&sprefix=uncanny+x+forceomnibus%2Caps%2C361&sr=8-2'


In [7]:
legoURL = 'https://www.amazon.com.au/LEGO-76269-Avengers-Tower-Available/dp/B0CN78FNTY/ref=sr_1_1?crid=25473F843JZ4S&dib=eyJ2IjoiMSJ9.Z6TJs4L31UU2mYyvLIULxf7aTn6Zt5ziJMo_NGus07COiKZSdET5ykByB5blJAwqvVUtQJ9WX-e3a4C5IIUobChlEeStEiK1AQzlbqQ7cEpN8K4_LDIYTZLhvsKd4OezKtmNXHGXm3nQzPxk_oApwdGknMXfRpX2jwnaowgRsqO4nVslMiTq-gV5mRkHNNSv2wfM3RbV4zx1J8XACTu9gWzC_oUsTCe0FyNd3IlxtKlFkgd2OTN9ICeyARLlWuTy8BKkjU9fneVQ3Er-gUvvjgVEeiYv3hi734MSgQ3otV8.2JPLVzNHxvI1u6Vl6A6b8SWjh5y2KT9xJi9NPFabnt0&dib_tag=se&keywords=lego+avengers+tower&qid=1722499021&sprefix=lego+avengers+tow%2Caps%2C295&sr=8-1'
